In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 24 13:02:32 2021

@author: lenovo
"""
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.io
import math
import matplotlib.gridspec as gridspec
from plotting import newfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import layers, activations
from scipy.interpolate import griddata
from eager_lbfgs import lbfgs, Struct
from pyDOE import lhs

from generate_dataset import *
from utilities import *

OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0-15
OMP: Info #216: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #157: KMP_AFFINITY: 16 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Info #287: KMP_AFFINITY: topology layer "LL cache" is equivalent to "socket".
OMP: Info #287: KMP_AFFINITY: topology layer "L3 cache" is equivalent to "socket".
OMP: Info #287: KMP_AFFINITY: topology layer "L2 cache" is equivalent to "core".
OMP: Info #287: KMP_AFFINITY: topology layer "L1 cache" is equivalent to "core".
OMP: Info #192: KMP_AFFINITY: 1 socket x 8 cores/socket x 2 threads/core (8 total cores)
OMP: Info #218: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #172: KMP_AFFINITY: OS proc 0 maps to socket 0 core 0 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 8 maps to socket 0 core 0 thread 1 
OMP: Info #172: KMP_AFFINITY: OS proc 1 maps to socket 0 core 1 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 9 maps to socket 0 core 1 thread 1 
OMP: Info

In [2]:
class neural_net(object):

    def __init__(self, layers):

        self.layer_sizes = layers
        sizes_w = []
        sizes_b = []
        for i, width in enumerate(self.layer_sizes):
            if i != 1:
                sizes_w.append(int(width * self.layer_sizes[1]))
                sizes_b.append(int(width if i != 0 else self.layer_sizes[1]))


        # L-BFGS weight getting and setting from https://github.com/pierremtb/PINNs-TF2.0

    def set_weights(self, model, w, sizes_w, sizes_b):  # 重新设置参数

        for i, layer in enumerate(model.layers[1:len(sizes_w) + 1]):
            start_weights = sum(sizes_w[:i]) + sum(sizes_b[:i])
            end_weights = sum(sizes_w[:i + 1]) + sum(sizes_b[:i])
            weights = w[start_weights:end_weights]
            w_div = int(sizes_w[i] / sizes_b[i])
            weights = tf.reshape(weights, [w_div, sizes_b[i]])
            biases = w[end_weights:end_weights + sizes_b[i]]
            weights_biases = [weights, biases]
            layer.set_weights(weights_biases)


    def get_weights(self, model):
        w = []
        for layer in model.layers[1:len(sizes_w) + 1]:
            weights_biases = layer.get_weights()
            weights = weights_biases[0].flatten()
            biases = weights_biases[1]
            w.extend(weights)
            w.extend(biases)
        w = tf.convert_to_tensor(w)
        return w

    def xavier_init(self, layer_sizes):
        in_dim = layer_sizes[0]
        out_dim = layer_sizes[1]
        xavier_stddev = np.sqrt(2 / (in_dim + out_dim))
        return tf.Variable(tf.random.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)

    def model(self):

        input_tensor = keras.Input(shape=(self.layer_sizes[0],))

        hide_layer_list = []
        flag = True
        for width in self.layer_sizes[1:-1]:
            if flag:
                x = layers.Dense(
                    width, activation=tf.nn.tanh,
                    kernel_initializer="glorot_normal")(input_tensor)
                flag = False
            else:
                x = layers.Dense(
                    width, activation=tf.nn.tanh,
                    kernel_initializer="glorot_normal")(x)
        output_tensor = layers.Dense(self.layer_sizes[-1], activation=None,kernel_initializer="glorot_normal")(x)
        print("xxxxxxxxxxxxxx")
        output0 = output_tensor[:, 0:1]
        output1 = output_tensor[:, 1:2]
        output2 = output_tensor[:, 2:3]

        model_output = keras.models.Model(input_tensor, [output0, output1, output2])
        model_output.summary()

        return model_output



In [3]:
class TwoD_BFS_Slip_PINN:

    def __init__(self, data, layers  , activFun , mode , starter_learning_rate = 1.0e-3 , ExistModel=0):


        self.layers = layers # 


        # initialize the NN
        self.net_cuvwp = neural_net(layers).model()
        self.weight_initial = []
        self.weightf = []

    @tf.function
    def f_model(self,t ,  x, y):

        mu = 0.00345
        density = 1056

        u, v, p = self.net_cuvwp(tf.concat([t, x , y],1))

        
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_y = tf.gradients(u, y)[0]
        u_xx = tf.gradients(u_x, x)[0]
        u_yy = tf.gradients(u_y, y)[0]

        v_t = tf.gradients(v, t)[0]
        v_x = tf.gradients(v, x)[0]
        v_y = tf.gradients(v, y)[0]
        v_xx = tf.gradients(v_x, x)[0]
        v_yy = tf.gradients(v_y, y)[0]

        p_x = tf.gradients(p, x)[0]
        p_y = tf.gradients(p, y)[0]

        f_u = u_t + (u * u_x + v * u_y ) + 1.0/density * p_x - mu *(u_xx + u_yy )
        f_v = v_t + (u * v_x + v * v_y ) + 1.0/density * p_y - mu *(v_xx + v_yy )
        div = u_x + v_y 
            

        return f_u, f_v, div


    # define the loss
    def loss_funct(self ,  t_f_batch , x_f_batch, y_f_batch, t_WALL_batch , x_WALL_batch, y_WALL_batch, p_WALL_batch ,
                    tb , xb, yb, ub, vb, pb, weight_WALL ,  weight_ub,  weight_fu):


        f_u_pred, f_v_pred, div_pred = self.f_model( t_f_batch , x_f_batch, y_f_batch)


        u_pred, v_pred, p_pred = self.net_cuvwp(tf.concat([tb , xb, yb], 1))
        u_WALL_pred, v_WALL_pred, _ = self.net_cuvwp(tf.concat([t_WALL_batch , x_WALL_batch, y_WALL_batch], 1))

        loss_INITIAL = weight_ub*(tf.reduce_mean(tf.square(u_pred - ub)) + tf.reduce_mean(tf.square(v_pred - vb))+ tf.reduce_mean(tf.square(p_pred - pb)))
        loss_WALL =   weight_WALL *  tf.reduce_mean(tf.square(u_WALL_pred ))  + 1.0* tf.reduce_mean(tf.square(v_WALL_pred))

        loss_res = weight_fu*(tf.reduce_mean(tf.square(f_u_pred)) + tf.reduce_mean(tf.square(f_v_pred)) + tf.reduce_mean(tf.square(div_pred)))

        return loss_WALL , loss_INITIAL, loss_res
    


    @tf.function
    def grad(self , t_f_batch , x_f_batch, y_f_batch, t_WALL_batch , x_WALL_batch, y_WALL_batch, p_WALL_batch , tb_batch ,  xb_batch, yb_batch, ub_batch, vb_batch, pb_batch, weight_ub, weight_fu):

        with tf.GradientTape(persistent=True) as tape:
            loss_WALL, loss_INITIAL, loss_res = self.loss_funct( t_f_batch , x_f_batch, y_f_batch, 
                                                           t_WALL_batch , x_WALL_batch, y_WALL_batch, p_WALL_batch , 
                                                           tb_batch , xb_batch, yb_batch , ub_batch, vb_batch , pb_batch, 
                                                           weight_ub, weight_fu)
            
            loss = loss_WALL + loss_INITIAL + loss_res
            grads = tape.gradient(loss, self.net_cuvwp.trainable_variables)

            grads_ub = tape.gradient(loss, weight_ub)

            grads_fu = tape.gradient(loss, weight_fu)

        return  loss_INITIAL, loss_res, grads, grads_ub, grads_fu



#model.train(collo[::10,:], inlet, outlet, wall, initial,  weight_ub, weight_fu, tf_iter=1000,   tf_iter2=100, newton_iter2=1500)
    def train( self , collo, inlet, outlet, wall, initial , weight_initial, weight_fu , tf_iter, tf_iter2, newton_iter2):

        batch_size =  120
        n_batches =  collo.shape[0] // batch_size

        start_time = time.time()

        tf_optimizer = tf.keras.optimizers.Adam(lr=0.005, beta_1=.99)
        tf_optimizer_weights = tf.keras.optimizers.Adam(lr=0.003, beta_1=.99)
        tf_optimizer_initial = tf.keras.optimizers.Adam(lr=0.03, beta_1=.99)

        tf.print(f"weight_initial: {weight_initial}  weight_fu: {weight_fu}")
        print("starting Adam training")

        a = np.random.rand(1000)
        loss_history = list(a)
        MSE_b0 = list(a)
        MSE_f0 = list(a)

        MSE_b1 = []
        MSE_f1 = []


        # For mini-batch (if used)
        for epoch in range(tf_iter):
            for i in range(n_batches):

                idx_WALL = np.random.choice(wall[:, 0:1].shape[0], np.min([wall[:, 0:1].shape[0],batch_size]), replace=False)

                xb_batch = initial[:, 1:2]
                yb_batch = initial[:, 2:3]
                tb_batch = initial[:, 0:1]
                ub_batch = initial[:, 3:4]
                vb_batch = initial[:, 4:5]
                pb_batch = initial[:, 5:6]

                (t_WALL_batch,
                x_WALL_batch,
                y_WALL_batch,
                p_WALL_batch) = (wall[:, 0:1][idx_WALL],
                                wall[:, 1:2][idx_WALL],
                                wall[:, 2:3][idx_WALL],
                                wall[:, 5:6][idx_WALL])
                
                x_f_batch = collo[:, 1:2][i * batch_size:(i * batch_size + batch_size), ]
                y_f_batch = collo[:, 2:3][i * batch_size:(i * batch_size + batch_size), ]
                t_f_batch = collo[:, 0:1][i * batch_size:(i * batch_size + batch_size), ]

                loss_value , grads, grads_ub, grads_fu = self.grad( t_f_batch , x_f_batch, y_f_batch, 
                                                                    t_WALL_batch , x_WALL_batch, y_WALL_batch, p_WALL_batch , 
                                                                      tb_batch , xb_batch, yb_batch , ub_batch, vb_batch, pb_batch,
                                                                          weight_initial, weight_fu)
                

                # print("starting Adam training")

                tf_optimizer.apply_gradients(zip(grads, self.net_cuvwp.trainable_variables))

                loss_history.append(loss_value)
                
                if loss_history[-1] < loss_history[-2] and loss_history[-2] < loss_history[-3] and loss_history[-1] < loss_history[-10]:
                    tf_optimizer_weights.apply_gradients(zip([-grads_fu], [weight_fu]))
                    tf_optimizer_initial.apply_gradients(zip([-grads_ub], [weight_initial]))

            if epoch % 10 == 0:
                elapsed = time.time() - start_time
                print('It: %d, Time: %.2f' % (epoch, elapsed))
                tf.print(f"mse_b  {mse_b}  mse_f: {mse_f}   total loss: {loss_value}")

                wu = weight_initial.numpy()
                wf = weight_fu.numpy()

                MSE_b1.append(mse_b)
                MSE_f1.append(mse_f)



                start_time = time.time()
    

        return MSE_b1, MSE_f1,  weightu, weightf


    # L-BFGS implementation from https://github.com/pierremtb/PINNs-TF2.0
    def get_loss_and_flat_grad ( self , t_f_batch , x_f_batch, y_f_batch, xb_batch, yb_batch, tb_batch, ub_batch, vb_batch,weight_ub, weight_fu):
        def loss_and_flat_grad(w):
            with tf.GradientTape() as tape:
                self.net_cuvwp.set_weights( w, self.net_cuvwp.sizes_w, self.net_cuvwp.sizes_b)
                loss_value, _, _ = self.loss_funct(x_f_batch, y_f_batch, t_f_batch, xb_batch, yb_batch, tb_batch, ub_batch, vb_batch, weight_ub, weight_fu)
            grad = tape.gradient(loss_value, self.net_cuvwp.trainable_variables)
            grad_flat = []
            for g in grad:
                grad_flat.append(tf.reshape(g, [-1]))
            grad_flat = tf.concat(grad_flat, 0)
            # print(loss_value, grad_flat)
            return loss_value, grad_flat

        return loss_and_flat_grad


    def predict(self , X_star):
        X_star = tf.convert_to_tensor(X_star, dtype=tf.float32)

        u, v, p = self.net_cuvwp(tf.concat([ X_star[:, 0:1] , X_star[:, 1:2], X_star[:, 2:3]], 1))

        return u.numpy(), v.numpy(), p.numpy()


In [4]:
########################################################################################

def get_training_dataset(path , pINLET , pOUTLET , pWALL , pDomain , pInitial , dist):
    
    data = h5py.File(path , 'r')  # load dataset from matlab
    WALL = np.transpose(data['noslipwall_02_Z0slice_2D_wall'], axes=range(len(data['noslipwall_02_Z0slice_2D_wall'].shape) - 1,-1, -1)).astype(np.float32)
    
    WALL = np.delete(WALL , np.where(WALL[:,0] == WALL[:,0].min())[0] , 0)  
    # WALL = np.delete(WALL , np.where(WALL[:,1] <= 0.2)[0] , 0)  


    domain = np.transpose(data['noslipwall_02_Z0slice_2D_innerdomain'], axes=range(len(data['noslipwall_02_Z0slice_2D_innerdomain'].shape) - 1,-1, -1)).astype(np.float32)
    
    domain = np.delete(domain , np.where(domain[:,0] == domain[:,0].min())[0] , 0)  
    # domain = np.delete(domain , np.where(domain[:,1] <= 0.2)[0] , 0)  


    # INLET = np.transpose(data['noslipwall_02_Z0slice_2D_inlet'],  axes=range(len(data['noslipwall_02_Z0slice_2D_inlet'].shape) - 1,-1, -1)).astype(np.float32)
        
    INLET = domain[np.where(domain[:,1] == domain[:,1].min())[0],:] #np.delete(INLET , np.where(INLET[:,0] == INLET[:,0].min())[0] , 0)  

    OUTLET = np.transpose(data['noslipwall_02_Z0slice_2D_outlet'], axes=range(len(data['noslipwall_02_Z0slice_2D_outlet'].shape) - 1,-1, -1)).astype(np.float32)
    
    OUTLET = np.delete(OUTLET , np.where(OUTLET[:,0] == OUTLET[:,0].min())[0] , 0)  

    total = INLET.shape[0] + OUTLET.shape[0] + domain.shape[0] +  WALL.shape[0] 
    
    np.random.seed(1234)

    # initial domain ux is 0.2 and other values are zero. FOr wall, all values are zero
    
    INITIALd = domain[np.where(domain[:,0] == domain[:,0].min())[0],:] # initial doamin corressponds to all values where t is zero

    INITIALw = WALL[np.where(WALL[:,0] == WALL[:,0].min())[0],:] # initial doamin corressponds to all values where t is zero

    INITIALi = INLET[np.where(INLET[:,0] == INLET[:,0].min())[0],:] # initial doamin corressponds to all values where t is zero

    INITIALo = OUTLET[np.where(OUTLET[:,0] == OUTLET[:,0].min())[0],:] # initial doamin corressponds to all values where t is zero

    #random selection of training data
    INITIAL = np.concatenate([INITIALd , INITIALo],0)
    
    # domain = np.concatenate([domain , WALL , INLET , OUTLET],0)

    #INLET = np.delete(INLET , idx_initi , 0)  
    #OUTLET = np.delete(OUTLET , idx_inito  , 0)  
    #domain = np.delete(domain , idx_initd , 0)  
    #WALL = np.delete(WALL , idx_initw , 0)  
    
    if dist == "Sobol":
        idxi = generate_sobol_sequence(0 , INLET.shape[0] ,  int(INLET.shape[0] * pINLET)) 
        INLET = INLET[idxi, :]
        idxi = generate_sobol_sequence(0 , OUTLET.shape[0] ,  int(OUTLET.shape[0] * pOUTLET)) 
        OUTLET = OUTLET[idxi, :]
        idxi = generate_sobol_sequence(0 , INITIAL.shape[0] ,  int(INITIAL.shape[0] * pInitial)) 
        INITIAL = INITIAL[idxi, :]
        idxi = generate_sobol_sequence(0 , WALL.shape[0] ,  int(WALL.shape[0] * pWALL)) 
        WALL = WALL[idxi, :]
        idxi = generate_sobol_sequence(0 , domain.shape[0] ,  int(domain.shape[0] * pDomain)) 
        domain = domain[idxi, :]
    else:
        idxi = np.random.choice(INLET.shape[0], int(INLET.shape[0] * pINLET), replace=False)
        INLET = INLET[idxi, :]
        idxi = np.random.choice(OUTLET.shape[0], int(OUTLET.shape[0] * pOUTLET), replace=False)
        OUTLET = OUTLET[idxi, :]
        idxi = np.random.choice(INITIAL.shape[0], int(INITIAL.shape[0] * pInitial), replace=False)
        INITIAL = INITIAL[idxi, :]
        idxi = np.random.choice(WALL.shape[0], int(WALL.shape[0] * pWALL), replace=False)
        WALL = WALL[idxi, :]
        idxi = np.random.choice(domain.shape[0], int(domain.shape[0] * pDomain), replace=False)
        domain = domain[idxi, :]

    #########################################
    return [domain , INLET , OUTLET, WALL, INITIAL , total]
################################################################



def plot_result(model ,path ):


    ######################################
    model.save_NN()

    list_ = [ "noslipwall_02_Z0slice_2D_wall", "noslipwall_02_Z0slice_2D_innerdomain" ]
    N_data = [ 1600 , 20800 ]

    peotDic = dict((key,value) for key,value in zip(list_,N_data))

    tstep = [ 100 , 100 , ]
    tstepList = dict((key,value) for key,value in zip(list_,tstep))

    for key,value in peotDic.items():
        # print(key, value)
        test_data_inlet = get_testing_dataset(path   , part=key)
        test_data_inlet = test_data_inlet[np.argsort(test_data_inlet[:, 0])]

        [_ ,xf , _ , ufa , vfa  , pfa, u_pred , v_pred, p_pred]  = predict_result(model , test_data_inlet ,  value ,  tstep  ,  text=key , stm = False)
        peotDic[key] =  error_over_time(tstepList[key] ,value , ufa , vfa , pfa , u_pred , v_pred , p_pred )
        l1l2Erorr(key , u_pred ,v_pred , p_pred , ufa , vfa , pfa , model)

    draw_error_over_time(peotDic , model.dirname)

        ## drawing velocity profile
    xValues = [5.000e-04 , 0.2275, 9.995e-01]
    # UVelocity_profile(model.dirname , 0.2 , xValues , xf , u_pred)

    
    #def predict_result(model , test_data , N_data , tstep  , text = 'all'  , stm = False):
    part = 'noslipwall_02_Z0slice_2D_innerdomain'
    test_data_innerdomain = get_testing_dataset(path    ,  part=part)
    [tf , xf , yf , ufa , vfa  , pfa, u_pred , v_pred, p_pred]   = predict_result(model , test_data_innerdomain , N_data ,  tstep ,  text='domain' ,  stm = False)

    

    tstep =100
    N_data = 20800
    x = xf.reshape(tstep,N_data)[0,:]
    y = yf.reshape(tstep,N_data)[0,:]
    t = tf.reshape(tstep,N_data)[:,0].T
    ufa = ufa.reshape(tstep,N_data)
    vfa = vfa.reshape(tstep,N_data)
    pfa = pfa.reshape(tstep,N_data)
    u_pred = u_pred.reshape(tstep,N_data)
    v_pred = v_pred.reshape(tstep,N_data)
    p_pred = p_pred.reshape(tstep,N_data)
    error_u =  np.abs(ufa - u_pred) # (u - u_pred) / u # np.abs(u - u_pred)
    error_v =  np.abs(vfa - v_pred) # (v - v_pred) / v # np.abs(v - v_pred)
    error_p =  np.abs(pfa - p_pred) # (p - p_pred) / p # np.abs(p - p_pred)

    data = [u_pred, v_pred , p_pred , ufa, vfa , pfa , error_u ,  error_v ,error_p ]

    plot_time_profile(model.dirname , x , y , t , ufa , u_pred , '$u$')
    plot_time_profile(model.dirname , x , y , t , vfa , v_pred , '$v$')
    plot_time_profile(model.dirname , x , y , t , pfa , p_pred , '$p$')

    draw_contourf(t , x , y , data , 1.0 ,10 , model.dirname , 5 , fontsize=17.5 , labelsize=12.5 , axes_pad=1.2)

    model.weight_change_per_layer()



In [5]:
from generate_dataset import * 
# path = '/okyanus/users/afarea/dataDir/noslipwall_02_Z0slice_2D/noslipwall_02_Z0slice_2D.mat'
path = '/media/afrah2/MyWork/files2023/dataset/noslipwall_02_Z0slice_2D.mat'

modelPath = ''


pINLET = 1
pOUTLET= 0.3
pWALL =  0.006
pDomain = 0.0007
pInitial = 0.07

dist =  "Sobol"
activFun = 'hard_swish'

[collo , inlet , outlet, wall, initial , total] = get_training_dataset(path , pINLET , pOUTLET , pWALL , pDomain , pInitial , dist)

XY_c = np.concatenate([collo , wall, inlet , outlet], 0) 



# # Define model
mode = 'M1'

input_dimension = 3
output_dimension = 3
n_hidden_layers = 3
neurons = 20 # [3, 20, 20, 20, 20, 20, 20, 20, 3]

starter_learning_rate = float(0.005)
epochs = 5
model_layers = [input_dimension] + n_hidden_layers*[neurons] + [output_dimension]

batch_size= 300

iterations = 40000

method  = "mini_batch"


###############################################################
model = TwoD_BFS_Slip_PINN(XY_c , model_layers  , activFun , mode , starter_learning_rate , ExistModel = modelPath)


# model.print("Using mode: " , model.mode)
# model.print("neural network: " , model.layers )
# model.print("collocation Training data size : " ,collo.shape[0], " (" ,str(pDomain*100) , "%)")
# model.print("Wall Training data size: " ,wall.shape[0], " (" ,str(pWALL*100) , "%)")
# model.print("Initial Training data size: " ,initial.shape[0], " (" ,str(pInitial*100) , "%)")
# model.print("INLET Training data size: " ,inlet.shape[0], " (" ,str(pINLET*100) , "%)")
# model.print("OUTLET Training data size: " ,outlet.shape[0], " (" ,str(pOUTLET*100) , "%)")

# model.print("Total training datset size: " ,XY_c.shape[0], " (" ,str((XY_c.shape[0] / total)*100) , "%)")
# model.print("Activation function: " , activFun)
# model.print("number of iterations: " , iterations)

# model.print("Method desciption : learning rates: Exponential decay  with initial value: ", starter_learning_rate)


# model.print("File directory: " , model.dirname)

# #save_data_to_matlab(os.path.join(model.dirname,'slipwall_02_Z0slice_2D_training.mat') ,coll , INLET , OUTLET , WALL ,INITIAL )

# plot_dataset( model.dirname , wall , inlet , outlet , collo , initial , dist)




xxxxxxxxxxxxxx
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 20)           80          input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 20)           420         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 20)           420         dense_1[0][0]                    
_______________________________________________________________________________

In [6]:

X_f = XY_c
X_u_train = wall



X_star1 = collo


b = X_star1.min(0)
rb = X_star1.max(0)

start_time = time.time()

In [7]:
N_f = 10000
Nu1 = 200

weight_initial = tf.Variable([1.0], dtype=tf.float32)
weight_fu = tf.Variable([1.0], dtype=tf.float32)

MSE_b1, MSE_f1, weightu, weightf = model.train(collo[::10,:], inlet, outlet, wall, initial,  weight_initial, weight_fu, tf_iter=1000,   tf_iter2=100, newton_iter2=1500)




weight_ub: <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([1.], dtype=float32)>  weight_fu: <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([1.], dtype=float32)>
starting Adam training


OMP: Info #254: KMP_AFFINITY: pid 25589 tid 25616 thread 1 bound to OS proc set 1
OMP: Info #254: KMP_AFFINITY: pid 25589 tid 25637 thread 2 bound to OS proc set 2
OMP: Info #254: KMP_AFFINITY: pid 25589 tid 25640 thread 5 bound to OS proc set 5
OMP: Info #254: KMP_AFFINITY: pid 25589 tid 25639 thread 4 bound to OS proc set 4
OMP: Info #254: KMP_AFFINITY: pid 25589 tid 25638 thread 3 bound to OS proc set 3
OMP: Info #254: KMP_AFFINITY: pid 25589 tid 25641 thread 6 bound to OS proc set 6
OMP: Info #254: KMP_AFFINITY: pid 25589 tid 25642 thread 7 bound to OS proc set 7
OMP: Info #254: KMP_AFFINITY: pid 25589 tid 25643 thread 8 bound to OS proc set 8
OMP: Info #254: KMP_AFFINITY: pid 25589 tid 25644 thread 9 bound to OS proc set 9
OMP: Info #254: KMP_AFFINITY: pid 25589 tid 25645 thread 10 bound to OS proc set 10
OMP: Info #254: KMP_AFFINITY: pid 25589 tid 25646 thread 11 bound to OS proc set 11
OMP: Info #254: KMP_AFFINITY: pid 25589 tid 25647 thread 12 bound to OS proc set 12
OMP: Info 

It: 0, Time: 14.46
mse_b  [30155.94]  mse_f: [17.521324]   total loss: [30173.46]
It: 10, Time: 0.43
mse_b  [4424.8906]  mse_f: [4.8558607]   total loss: [4429.7466]
It: 20, Time: 0.27
mse_b  [4600.787]  mse_f: [4.5006948]   total loss: [4605.2876]
It: 30, Time: 0.98
mse_b  [966.13116]  mse_f: [4.3892517]   total loss: [970.5204]
It: 40, Time: 0.66
mse_b  [2345.0269]  mse_f: [3.0802488]   total loss: [2348.1072]
It: 50, Time: 0.25
mse_b  [2062.1064]  mse_f: [3.9237363]   total loss: [2066.0303]
It: 60, Time: 0.77
mse_b  [1244.8021]  mse_f: [3.7409902]   total loss: [1248.5431]
It: 70, Time: 0.26
mse_b  [1521.5153]  mse_f: [2.2677171]   total loss: [1523.783]
It: 80, Time: 0.71
mse_b  [874.7628]  mse_f: [1.0880482]   total loss: [875.8509]
It: 90, Time: 0.35
mse_b  [1184.888]  mse_f: [0.6953928]   total loss: [1185.5834]
It: 100, Time: 0.67
mse_b  [1173.1166]  mse_f: [0.76007694]   total loss: [1173.8767]
It: 110, Time: 0.29
mse_b  [642.6582]  mse_f: [0.8639137]   total loss: [643.5221]

In [17]:
key = "noslipwall_02_Z0slice_2D_innerdomain" 

test_data_innerdomain = get_testing_dataset(path   , part=key)


In [28]:
test_data_innerdomain[:, 1:2].shape

test = ([test_data_innerdomain[:, 1:2] , test_data_innerdomain[:, 2:3] , test_data_innerdomain[:, 0:1]])
len(test)

3

In [29]:
tf.print(f"weight_ub: {weight_initial}  weight_fu: {weight_fu}")
u_pred, v_pred, p_pred = model.predict(test_data_innerdomain[:,  0:3])



# lbfgs(loss_and_flat_grad, get_weights(u_model), Struct(), maxIter=newton_iter2, learningRate=0.8)

weight_ub: <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([3.8362556], dtype=float32)>  weight_fu: <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([1.2829995], dtype=float32)>
Error u: 5.784612e+02
Error v: 1.589013e+04
Starting L-BFGS training


NameError: name 'x_f_batch' is not defined

In [35]:
error_u = np.linalg.norm(test_data_innerdomain[:,  3:4] - u_pred, 2) / np.linalg.norm(test_data_innerdomain[:,  3:4] , 2)
print('Error u: %e' % (error_u))
error_v = np.linalg.norm(test_data_innerdomain[:,  4:5] - v_pred, 2) / np.linalg.norm(test_data_innerdomain[:,  4:5], 2)
print('Error v: %e' % (error_v))
error_v = np.linalg.norm(test_data_innerdomain[:,  5:6] - v_pred, 2) / np.linalg.norm(test_data_innerdomain[:,  5:6], 2)
print('Error v: %e' % (error_v))
print("Starting L-BFGS training")


Error u: 4.151636e-01
Error v: 1.034749e+02
Error v: 9.199123e-01
Starting L-BFGS training


In [34]:
loss_and_flat_grad = get_loss_and_flat_grad(t_star1 , x_star1, y_star1, tb ,  xb, yb,  ub,  vb , weight_initial, weight_fu)

lbfgs(loss_and_flat_grad,  get_weights(u_model), Struct(), maxIter = 500, learningRate=0.8)

u_pred, v_pred, p_pred = predict(test_data_innerdomain[:,  0:3])
error_u = np.linalg.norm(u_exact1 - u_pred, 2) / np.linalg.norm(u_exact1, 2)
print('Error u: %e' % (error_u))
error_v = np.linalg.norm(v_exact1 - v_pred, 2) / np.linalg.norm(v_exact1, 2)
print('Error v: %e' % (error_v))

error_p = np.linalg.norm(p_exact1 - p_pred, 2) / np.linalg.norm(p_exact1, 2)
print('Error p: %e' % (error_p))


TypeError: loss_funct() missing 1 required positional argument: 'weight_fu'

In [16]:

elapsed = time.time() - start_time
print('Training time: %.4f' % (elapsed))

u_pred, v_pred, p_pred = predict(X_star1)

# U_pred = u_pred.reshape((x1.shape[0], y1.shape[0], t1.shape[0]))
# V_pred = v_pred.reshape((x1.shape[0], y1.shape[0], t1.shape[0]))
# P_pred = p_pred.reshape((x1.shape[0], y1.shape[0], t1.shape[0]))
u_exact1 = np.array([X_star1[:, 3]])
v_exact1 = np.array([X_star1[:, 4]])
p_exact1 = np.array([X_star1[:, 5]])


error_uu = np.abs( u_exact1 - u_pred)
error_vv = np.abs(v_exact1 - v_pred)
error_pp = np.abs( p_exact1 - p_pred)

error_u = np.linalg.norm(u_exact1 - u_pred, 2) / np.linalg.norm(u_exact1, 2)
print('Error u: %e' % (error_u))

error_v = np.linalg.norm(v_exact1 - v_pred, 2) / np.linalg.norm(v_exact1, 2)
print('Error v: %e' % (error_v))

error_p = np.linalg.norm(p_exact1 - p_pred, 2) / np.linalg.norm(p_exact1, 2)
print('Error p: %e' % (error_p))

# dataNewNS = 'NS_hisyory.mat'
# scipy.io.savemat(dataNewNS, {'w_MSE_b': MSE_b1, 'w_MSE_f': MSE_f1, 'weight_u': weightu,
#                   'weight_f': weightf, 'U_pred': u_pred, 'V_pred': V_pred, 'P_pred': P_pred})

Training time: 1488.1751
Error u: 4.266133e+01
Error v: 4.266161e+01
Error p: 8.245512e+01
